In [76]:
from pathlib import Path
import os
import re
from bs4 import BeautifulSoup as bs

In [86]:
def melon_parser():
    # top 100 페이지 전처리

    current_folder = Path.cwd() # 현재 폴더
    parent_folder = current_folder.parent # 상위 폴더 이동
    melon_folder = os.path.join(str(parent_folder) + '\\assets\data\stage1\scrapper\melon') # 크롤링한 Html 폴더 이동
    Path(f'{melon_folder}\\result').mkdir(parents=True, exist_ok=True) # 결과 폴더 생성

    ranking_data_html = os.listdir(melon_folder)[:-1] # result 폴더 제외한 html파일

    for genre_rank_html in ranking_data_html: # 장르별 html파일 처리
        if genre_rank_html.find(".html") != -1: # html 파일만 처리
            with open(f'{melon_folder}\{genre_rank_html}', 'r') as file:
                soup = bs(file, 'html.parser')
                top100 = soup.find_all("tr", {"class": "lst50"}) + soup.find_all("tr", {"class": "lst100"}) # 100위까지 정보 합침
                
                # csv 파일명 포매팅
                file_name = re.search(r'\d{4}-\d{2}-\d{2}', genre_rank_html).group() # 날짜부분 추출
                genre_name = re.search(r'([a-zA-Z&]+)\.html$', genre_rank_html).group(1) # 장르부분 추출
                
                with open(f'{melon_folder}\\result\{file_name}_{genre_name}.csv', 'w') as f:
                    f.write("genre,rank,sondid,songname,artistid,artistname,albumid,albumname\n")
                    for row in top100:
                        rank = row.find("span", {"class": "rank"}).text.strip()
                        songid = row.attrs['data-song-no']
                        songname = f'\"{row.find("div", {"class": "ellipsis rank01"}).text.strip()}\"'
                        artistid = re.sub(r'[^0-9]', '', row.find("div", {"class": "ellipsis rank02"}).find("a").attrs['href'])
                        artistname = row.find("div", {"class": "ellipsis rank02"}).find("a").text.strip()
                        albumid = re.sub(r'[^0-9]', '', row.find("div", {"class": "ellipsis rank03"}).find("a").attrs['href'])
                        albumname = f'\"{row.find("div", {"class": "ellipsis rank03"}).text.strip()}\"'
                        f.write(f'{genre_name},{rank},{songid},{songname},{artistid},{artistname},{albumid},{albumname}\n')
                  
    # top 100에 등록된 곡 상세정보 전처리

    top100_folder_path = os.path.join(melon_folder) + '\\top100' # c:\Users\Harvey\Desktop\Codes\Python\NLP\LyricToImage\assets\data\stage1\scrapper\melon\top100
    top100_folders = os.listdir(top100_folder_path) # ['ballad', 'dance', 'ost', 'pop', 'rap&hiphop', 'trot']


    for genre_folder in top100_folders: # 장르별 처리
        song_data_htmls = os.listdir(os.path.join(top100_folder_path) + f"\{genre_folder}") # Html 파일 검색

        with open(f'{melon_folder}\\result\{file_name}_{genre_folder}_songdata.csv', 'w') as f:
            # f.write("genre,sondid,songname,artistid,artistname,albumid,albumname,release,flac,lyric,lyricist,composer,arranger\n")
            f.write("genre,sondid,songname,artistid,artistname,albumid,albumname,release,flac,lyric\n")
            for song_data_html in song_data_htmls: # Html 파일별 처리
                song_data_path = os.path.join(top100_folder_path + f'\{genre_folder}\{song_data_html}')

                # Html 문서 읽기
                with open(song_data_path, 'r') as file:
                    d_soup = bs(file, 'html.parser')
                    # 음악 id
                    d_songid = d_soup.find("button", {"class": "button_etc like type02"}).attrs['data-song-no'] 
                    
                    # 제목
                    d_songname_tag = d_soup.find("div", {"class": "song_name"})
                    strong_tag = d_songname_tag.find("strong")
                    strong_tag.decompose()
                    d_songname = f'\"{d_songname_tag.text.strip()}\"'
        
                    # 아티스트 id, 이름
                    d_artistid_n_name = d_soup.find("a", {"class": "artist_name"}).attrs 
                    d_artistid = re.sub(r'[^0-9]', '', d_artistid_n_name['href']) 
                    d_artistname = f'\"{d_artistid_n_name["title"]}\"'
                    
                    # 앨범 id, 앨범 제목, 발매일, 장르, FLAC 정보
                    metadata = d_soup.find("dl", {"class": "list"}).find_all("dd")
                    datum = []
                    for idx, data in enumerate(metadata):
                        if idx == 0:
                            a_id = data.find("a").attrs['href']
                            a_id = re.sub(r'[^0-9]', '', a_id)
                            datum.append(a_id)
                        datum.append(data.text.strip())    
                    d_albumid, d_albumname, d_releasedate, d_genre  = datum[0], f'\"{datum[1]}\"', datum[2], f'\"{datum[3]}\"'
                    if len(datum) == 5:
                        d_flactype = datum[4]
                    else:
                        d_flactype = ""
                    # print(*datum)
                    
                    # 가사 정보
                    lyric = d_soup.find("div", {"class": "lyric"}).text.replace("\n", " ").strip()
                    lyric = lyric.replace(",", "").strip()
                    lyric = re.sub(r"\s+", " ", lyric)
                    lyric = f'\"{lyric}\"'
                    
                    # # 작사가, 작곡가, 편곡가 정보
                    # l_c_a_data = []
                    # l_c_a_s = d_soup.find("ul", {"class": "list_person clfix"}).find_all("li")
                    # for l_c_a in l_c_a_s:
                    #     l_c_a_data.append(l_c_a.find("div", {"class": "ellipsis artist"}).text.strip())
                    
                    # d_lyricist, d_composer = l_c_a_data[0], l_c_a_data[1]
                    # if len(l_c_a_data) == 3:
                    #     d_arranger = l_c_a_data[2]
                    # else:
                    #     d_arranger = ""
                        
                f.write(f"{d_genre},{d_songid},{d_songname},{d_artistid},{d_artistname},{d_albumid},{d_albumname},{d_releasedate},{d_flactype},{lyric}\n")
                # f.write(f"{d_genre},{d_songid},{d_songname},{d_artistid},{d_artistname},{d_albumid},{d_albumname},{d_releasedate},{d_flactype},{lyric},{d_lyricist},{d_composer},{d_arranger}\n")
                
            

In [87]:
melon_parser()

In [ ]:
def genie_parer():

In [ ]:
def bugs_parser()